In [ ]:
import csv
from datetime import datetime

def load_transactions(filename='financial_transactions.csv', error_log='error_log.txt'):
    transactions = []
    try:
        with open(filename) as csvfile:
            reader = csv.DictReader(csvfile)
            for row_number, row in enumerate(reader, start=1):  # Track row number
                try:
                    ## Validate and convert date
                    date_str = row.get('date', '').strip()
                    if not date_str:
                        raise ValueError(f"Missing date field in row {row_number}")
                    try:
                        # Attempt to parse date in multiple formats
                        try:
                            date_converted = datetime.strptime(date_str, '%m/%d/%Y')  # MM/DD/YYYY
                        except ValueError:
                            date_converted = datetime.strptime(date_str, '%Y-%m-%d')  # YYYY-MM-DD
                        row['date'] = date_converted.strftime('%Y-%m-%d')  # Convert to standardized format
                    except ValueError:
                        raise ValueError(f"Invalid date format in row {row_number}: {date_str}. Expected MM/DD/YYYY or YYYY-MM-DD.")

                    # Validate and convert amount
                    amount_str = row.get('amount', '').strip()
                    if not amount_str:
                        raise ValueError(f"Missing amount field in row {row_number}")
                    if row.get('type', '').strip().lower() == 'debit':
                        row['amount'] = -abs(float(amount_str))
                    else:
                        row['amount'] = float(amount_str)

                    # Append valid transaction
                    transactions.append(row)
                except ValueError as e:
                    # Log error to file
                    with open(error_log, 'a') as log_file:
                        log_file.write(f"Error in row {row_number}: {e}\n")
    except FileNotFoundError:
        print("File not found.")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
    return transactions

def add_transaction(transactions):
    """Add a new transaction from user input."""
    new_transaction = {}
    # Date
    while True:
        date_input = input("Enter date (MM-DD-YYYY): ")
        try:
            datetime.strptime(date_input, '%m-%d-%Y')
            new_transaction['date'] = date_input
            break
        except ValueError:
            print("Invalid date format. Please use MM-DD-YYYY.")
    # Customer ID
    while True:
        customer_id_input = input("Enter customer ID: ")
        try:
            customer_id = int(customer_id_input)
            new_transaction['customer_id'] = customer_id
            break
        except ValueError:
            print("Customer ID must be a number.")
    # Amount
    while True:
        amount_input = input("Enter amount: ")
        try:
            amount = float(amount_input)
            new_transaction['amount'] = amount
            break
        except ValueError:
            print("Amount must be a number.")
    # Type
    while True:
        type_input = input("Enter type (credit/debit/transfer): ").lower()
        if type_input in ['credit', 'debit', 'transfer']:
            new_transaction['type'] = type_input
            break
        else:
            print("Type must be 'credit', 'debit', or 'transfer'.")
    # Description
    new_transaction['description'] = input("Enter description: ")
    if new_transaction['description'] == '':
        new_transaction['description'] = "No description provided."
    
    # Convert debit amounts to negative
    if new_transaction['type'] == 'debit':
        new_transaction['amount'] = -abs(new_transaction['amount'])
    # Generate transaction_id
    new_transaction['transaction_id'] = len(transactions) + 1
    
    transactions.append(new_transaction)
    print("Transaction added")
    pass

def view_transactions(transactions):
    """Display transactions in a table."""
    print(f"{'ID':<4} | {'Date':<10} | {'Customer':<8} | {'Amount':<8} | {'Type':<8} | {'Description'}")
    print("-" * 5 + "|" + "-" * 12 + "|" + "-" * 10 + "|" + "-" * 10 + "|" + "-" * 10 + "|" + "-" * 12)
    for transaction in transactions:
        print(f"{transaction['transaction_id']:<4} | {transaction['date']:<10} | {transaction['customer_id']:<8} | {transaction['amount']:<8.2f} | {transaction['type']:<8} | {transaction['description']}")
    pass

def update_transaction(transactions):
    """Update a transaction’s details."""
    print(f"{'ID':<4} | {'Date':<10} | {'Customer':<8} | {'Amount':<8} | {'Type':<8} | {'Description'}")
    print("-" * 5 + "|" + "-" * 12 + "|" + "-" * 10 + "|" + "-" * 10 + "|" + "-" * 10 + "|" + "-" * 12)
    for i, transaction in enumerate(transactions, start=1):
        print(f"{i:<4} | {transaction['date']:<10} | {transaction['customer_id']:<8} | {transaction['amount']:<8.2f} | {transaction['type']:<8} | {transaction['description']}")

    while True:
        try:
            transaction_number = int(input(f"Select transaction (1-{len(transactions)}): "))
            if transaction_number < 1 or transaction_number > len(transactions):
                print("Invalid transaction number. Please try again.")
                continue
            field_to_change = input("Change which field? (description, type, amount): ").lower()
            if field_to_change not in ['amount', 'type', 'description']:
                print("Invalid field. Please choose from description, type, or amount.")
                continue

            elif field_to_change == 'description':
                new_description = input("Enter new description: ")
                transactions[transaction_number - 1]['description'] = new_description if new_description else "No description provided."
            elif field_to_change == 'type':
                while True:
                    new_type = input("New type (credit/debit/transfer): ").lower()
                    if new_type in ['credit', 'debit', 'transfer']:
                        transactions[transaction_number - 1]['type'] = new_type
                        if new_type == 'debit':
                            transactions[transaction_number - 1]['amount'] = -abs(transactions[transaction_number - 1]['amount'])
                        else:
                            transactions[transaction_number - 1]['amount'] = abs(transactions[transaction_number - 1]['amount'])
                        break
                    else:
                        print("Type must be 'credit', 'debit', or 'transfer'.")
            elif field_to_change == 'amount':
                while True:
                    new_amount = input("Enter new amount: ")
                    try:
                        transactions[transaction_number - 1]['amount'] = float(new_amount)
                        if transactions[transaction_number - 1]['type'] == 'debit':
                            transactions[transaction_number - 1]['amount'] = -abs(transactions[transaction_number - 1]['amount'])
                        break
                    except ValueError:
                        print("Amount must be a number.")

            print("Transaction updated")
            break
        except ValueError:
            print("Invalid input. Please enter a valid transaction number.")
    pass

def delete_transaction(transactions):
    """Delete a transaction."""
    print(f"{'ID':<4} | {'Date':<10} | {'Customer':<8} | {'Amount':<8} | {'Type':<8} | {'Description'}")
    print("-" * 5 + "|" + "-" * 12 + "|" + "-" * 10 + "|" + "-" * 10 + "|" + "-" * 10 + "|" + "-" * 12)
    for i, transaction in enumerate(transactions, start=1):
        print(f"{i:<4} | {transaction['date']:<10} | {transaction['customer_id']:<8} | {transaction['amount']:<8.2f} | {transaction['type']:<8} | {transaction['description']}")

    while True:
        try:
            transaction_number = int(input(f"Select transaction (1-{len(transactions)}): "))
            if transaction_number < 1 or transaction_number > len(transactions):
                print("Invalid transaction number. Please try again.")
                continue
            confirm_deletion = input(f"Are you sure? (yes/no): ").lower()
            if confirm_deletion == 'yes':
                transactions.pop(transaction_number - 1)
                print("Transaction deleted")
                break
            elif confirm_deletion == 'no':
                print("Deletion cancelled")
                break
            else:
                print("Please answer with 'yes' or 'no'.")
        except ValueError:
            print("Invalid input. Please enter a valid transaction number.")
    pass

def get_financial_summary(transactions):
    total_income = 0.0
    total_expenses = 0.0
    total_transfers = 0.0
    balance_per_customer = {}

    for transaction in transactions:
        amount = transaction['amount']
        customer_id = transaction['customer_id']
        ttype = transaction['type']

        if ttype == 'credit':
            total_income += amount
        elif ttype == 'debit':
            total_expenses += abs(amount)
        elif ttype == 'transfer':
            total_transfers += abs(amount)

        if customer_id not in balance_per_customer:
            balance_per_customer[customer_id] = 0.0
        balance_per_customer[customer_id] += amount

    summary = []
    summary.append("Financial Summary:")
    summary.append(f"Total Credits: ${total_income:.2f}")
    summary.append(f"Total Debits: ${total_expenses:.2f}")
    summary.append(f"Total Transfers: ${total_transfers:.2f}")
    summary.append("Net Balance:")
    for customer_id, balance in balance_per_customer.items():
        summary.append(f"Customer {customer_id}: ${balance:.2f}")
    return "\n".join(summary)

def analyze_finances(transactions):
    """Calculate and display financial summaries."""
    print(get_financial_summary(transactions))

def save_transactions(transactions, filename='financial_transactions.csv'):
    """Save transactions to a CSV file without overwriting existing ones."""
    try:
        # Check if the file exists and read existing transactions
        existing_transactions = []
        try:
            with open(filename, 'r') as csvfile:
                reader = csv.DictReader(csvfile)
                existing_transactions = list(reader)
        except FileNotFoundError:
            pass  # File doesn't exist yet, proceed with writing new transactions

        # Combine existing transactions with new ones
        all_transactions = existing_transactions + transactions

        # Write all transactions to the file
        with open(filename, 'w', newline='') as csvfile:
            fieldnames = ['transaction_id', 'date', 'customer_id', 'amount', 'type', 'description']
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            writer.writeheader()
            for transaction in all_transactions:
                writer.writerow(transaction)

        print(f"Transactions saved to {filename}.")
    except Exception as e:
        print(f"An error occurred while saving transactions: {e}")
    pass

def generate_report(transactions, filename='report.txt'):
    """Generate a text report of financial summaries."""
    with open(filename, 'w') as f:
        f.write(get_financial_summary(transactions))
    print(f"Report written to {filename}.")
    pass

def main():
    transactions = []
    while True:
        print("\nSmart Personal Finance Analyzer")
        print("1. Load Transactions")
        print("2. Add Transaction")
        print("3. View Transactions")
        print("4. Update Transaction")
        print("5. Delete Transaction")
        print("6. Analyze Finances")
        print("7. Save Transactions")
        print("8. Generate Report")
        print("9. Exit")
        choice = input("Select an option: ")
        # Call functions based on choice
        if choice == '1':
            transactions = load_transactions()
            print("Transactions loaded!")
        elif choice == '2':
            add_transaction(transactions)
        elif choice == '3':
            view_transactions(transactions)
        elif choice == '4':
            update_transaction(transactions)
        elif choice == '5':
            delete_transaction(transactions)
        elif choice == '6':
            analyze_finances(transactions)
        elif choice == '7':
            save_transactions(transactions)
        elif choice == '8':
            generate_report(transactions)
        elif choice == '9':
            break
        else:
            print("Please choose a number between 1 and 9 to navigate.")
            pass

if __name__ == "__main__":
    main()


Smart Personal Finance Analyzer
1. Load Transactions
2. Add Transaction
3. View Transactions
4. Update Transaction
5. Delete Transaction
6. Analyze Finances
7. Save Transactions
8. Generate Report
9. Exit
Please choose a number between 1 and 9 to navigate.

Smart Personal Finance Analyzer
1. Load Transactions
2. Add Transaction
3. View Transactions
4. Update Transaction
5. Delete Transaction
6. Analyze Finances
7. Save Transactions
8. Generate Report
9. Exit
Please choose a number between 1 and 9 to navigate.

Smart Personal Finance Analyzer
1. Load Transactions
2. Add Transaction
3. View Transactions
4. Update Transaction
5. Delete Transaction
6. Analyze Finances
7. Save Transactions
8. Generate Report
9. Exit
Transactions loaded!

Smart Personal Finance Analyzer
1. Load Transactions
2. Add Transaction
3. View Transactions
4. Update Transaction
5. Delete Transaction
6. Analyze Finances
7. Save Transactions
8. Generate Report
9. Exit
Transactions loaded!

Smart Personal Finance Analyz